## Tutorial Taken from:
https://medium.com/analytics-vidhya/topic-modeling-using-gensim-lda-in-python-48eaa2344920

In [1]:
"""Import libraries"""
import nltk
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

nltk.download('stopwords')

In [2]:
"""Prepare stopwords"""

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
"""Load dataset"""
# LoadDataset
df=pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

In [4]:
"""Data cleaning:  Remove emails and newlines"""
# Convert to list 
data = df.content.values.tolist()  
# Remove Emails 
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]  
# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]  
pprint(data[:1])

In [5]:
"""Tokenization and cleaning up text"""
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [7]:
"""Remove stopwords, make bigrams, lemmatize"""
# Define function for stopwords, bigrams, trigrams and lemmatization

len_segments = 100
num_s

def remove_stopwords(texts):
    return_ls = []
    
    
    
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    return return_ls

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [9]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

In [10]:
"""Readable dictionary"""
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [11]:
"""Build topic model"""
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [13]:
"""Evaluate topic models"""
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [17]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/jcm/.local/lib/python3.10/site-packages/pyLDAvis/_prepare.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/jcm/.local/lib/python3.10/site-packages/pyLDAvis/_prepare.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.336524 -0.053704       1        1  34.413844
3      0.316929 -0.118895       2        1  15.585778
11     0.216787  0.285240       3        1   9.181672
6      0.247093  0.167004       4        1   9.110795
13     0.184104 -0.279652       5        1   7.614036
10    -0.019302  0.039633       6        1   5.391939
0      0.153912  0.027969       7        1   5.120545
9     -0.008544 -0.117709       8        1   2.140521
1      0.006923 -0.062099       9        1   2.065619
19     0.038373  0.069498      10        1   1.848057
16    -0.037562  0.037036      11        1   1.732310
4     -0.046033 -0.038083      12        1   1.606874
2     -0.081906  0.028034      13        1   1.237323
5     -0.118580  0.023843      14        1   1.144277
8     -0.162683  0.013122      15        1   0.619313
17    -0.175944 -0.007815      16        1   0.433673
15    -0.204259 -0.001773      17        1   0.294853
7     -0.212202 -0.003259      18        1   0.200450
14    -0.214082 -0.003627      19        1   0.158955
12    -0.219547 -0.004763      20        1   0.099166, topic_info=              Term          Freq         Total Category  logprob  loglift
4399            ax  53235.000000  53235.000000  Default  30.0000  30.0000
17            line  17060.000000  17060.000000  Default  29.0000  29.0000
208          write  13405.000000  13405.000000  Default  28.0000  28.0000
142   organization   9429.000000   9429.000000  Default  27.0000  27.0000
45         article   9896.000000   9896.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
4946            tv      0.020348      1.258244  Topic20 -11.0492   2.7917
6485   advertising      0.020348      1.258230  Topic20 -11.0492   2.7917
7775          logo      0.020348      1.258228  Topic20 -11.0492   2.7917
7567        circle      0.020348      1.258229  Topic20 -11.0492   2.7917
4391             _      0.020348      1.258254  Topic20 -11.0492   2.7916

[871 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
495       9  0.990578    absolute
1354      2  0.999137      accept
1277     15  0.843014  accessible
2283      6  0.995016  accomplish
183       1  0.170685     address
...     ...       ...         ...
40        8  0.002984        year
40        9  0.134080        year
614       1  0.590721       young
614       8  0.374635       young
614       9  0.031965       young

[1018 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 4, 12, 7, 14, 11, 1, 10, 2, 20, 17, 5, 3, 6, 9, 18, 16, 8, 15, 13])